## Part 2

1. Write a function that estimates the emission parameters `e(x|y)` from the training set using MLE (maximum likelihood estimation).

In [1]:
import utils
import emission

In [2]:
dataset_folder = "data/CN/"
train_data = dataset_folder + "train"
lines = utils.read_file_to_lines(train_data)

In [3]:
emission_data = emission.generate_emission_table(lines)

Skipped 1 lines:  ['']


2. Use label smoothing with `k=3` to account for rare and unknown words.

In [4]:
hashmap = emission_data["x_hashmap"]
og_vocab_size = len(hashmap.keys())
smoothed_hashmap = utils.add_unk(hashmap, k=3)
smooth_vocab_size = len(smoothed_hashmap.keys())
emission_data["x_hashmap"] = smoothed_hashmap
print("Reduced vocab from", og_vocab_size, "to", smooth_vocab_size)

Reduced vocab from 23766 to 5546


3. Implement a simple sentiment analysis system

In [5]:
eval_data = dataset_folder + "dev.in"
lines = utils.read_file_to_lines(eval_data)
outfile = dataset_folder + "dev.p2.out"

pos_tags = list(emission_data["y_tags"].keys())

preds = []

for word in lines:
    if word is not "":
        mle_list = [emission.get_mle(word, tag, emission_data) for tag in pos_tags]
        argmax_mle = pos_tags[mle_list.index(max(mle_list))]
        preds.append(argmax_mle)
    else:
        preds.append("")
    
output = zip(lines, preds)

with open(outfile, "w") as f:
    for line in output:
        f.write(line[0] + " " + line[1] +"\n")

In [6]:
!python3 evalResult.py ./data/CN/dev.out ./data/CN/dev.p2.out


#Entity in gold data: 1478
#Entity in prediction: 9741

#Correct Entity : 758
Entity  precision: 0.0778
Entity  recall: 0.5129
Entity  F: 0.1351

#Correct Sentiment : 265
Sentiment  precision: 0.0272
Sentiment  recall: 0.1793
Sentiment  F: 0.0472
